In [1]:
queriesData = []
queryStreaks = {}
banned = []
bannedKeywords = ['link:', '-host:', 'url:', 'like:', 'OR', 'AND']
humanUsers = [] 
humanQueries = 0

In [2]:
def printData(data):
    for item in data:
        print(item['id'])
        
def updateData(data, query, time):
    i = 0
    exists = False
    for item in data:
        if(item['id'] == query['id']):
            exists = True
            if time - item['timestamp'] < 30 and time - item['timestamp'] != 0:
                countStreak(item)
                countStreak(query)
            break
        i += 1
    if(exists):
        data[i] = query
    else:
        data.append(query)
    return data

def convertTimestamp(timestamp):
    temp = timestamp[11:]
    temp = temp.replace(":", "")
    return int(temp)
            
def cleanData(data, time):
    try:
        i = 0
        for item in data:
            if time - item['timestamp'] > 30:
                del data[i]
                i -= 1
            i += 1
    except IndexError as ex:
        print(ex)
    return data

def countStreak(query):
    streak = {}
    streak['text'] = query['text']
    streak['timestamp'] = query['timestamp']
    isBanned = True
    id = query['id']
    if(id not in banned):
        isBanned = False
    if(not isBanned):
        if(id in queryStreaks):
            queryStreaks[id].append(streak)
        else:
            queryStreaks[id] = [streak]
            
def buildChains(queryStreakData):
    tempList = []
    tempDict = {}
    
    currentTime = -1
    for item in queryStreakData:
        for query in queryStreakData[item]:
            if(currentTime == -1):
                currentTime = query['timestamp']
            if(abs(query['timestamp'] - currentTime) < 30):
                tempList.append(query['text'])
            else:
                if(item in tempDict):
                    tempDict[item].append(tempList)
                else:
                    tempDict[item] = [tempList]
                tempList = []
                tempList.append(query['text'])
                currentTime = query['timestamp']
    return tempDict


In [3]:
import codecs
file = codecs.open('data.txt', 'r','utf-8')
i = 0

for line in file:
    i += 1
    fields = line.split('\t')
    try:
        id = fields[0]
        text = fields[1]
        timestamp = fields[2]
        if any(word in text for word in bannedKeywords):
            banned.append(id)
        else:
            query = {}
            humanQueries += 1
            query['id'] = id
            query['text'] = text
            query['date'] = timestamp
            query['timestamp'] = convertTimestamp(timestamp)
            currentTime = query['timestamp']
            humanUsers.append(query['id'])
            queriesData = updateData(queriesData, query, currentTime)
        if(i % 500 == 0):
            queriesData = cleanData(queriesData, currentTime)
        if(i % 10000 == 0):
            print(i)
    except IndexError as er:
        print(i)
file.close()


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
127374
127375
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
137000

In [4]:
def deleteDuplicates(aList):
    seen = set()
    seen_add = seen.add
    return [x for x in aList if not (x in seen or seen_add(x))]
sortedHumanUsers = deleteDuplicates(humanUsers)

In [5]:
queryStreaks = buildChains(queryStreaks)

for item in queryStreaks:
    j = 0
    for streak in queryStreaks[item]:
        if(len(streak) <= 1):
            del queryStreaks[item][j]
        j += 1

In [12]:
print(len(queryStreaks))
print(humanQueries)

68431
2875887


In [8]:
import json
def export():
    with open('streaks.json', 'w') as outfile:  
        json.dump(queryStreaks, outfile)

In [9]:
export()

In [11]:
print(len(sortedHumanUsers))

366049
